In [ ]:
from kafka3 import KafkaProducer
from datetime import datetime
from time import sleep
from json import dumps
import datetime as dt
import random
import csv

In [ ]:
host_ip = "118.139.86.28"

In [ ]:
with open ("hotspot_TERRA_streaming.csv", "r") as file:
    TERRA_file = file.readlines()
    TERRA_file = [line.strip() for line in TERRA_file]
    for i in range(len(TERRA_file)):
        TERRA_file[i] = TERRA_file[i].split(",")

In [ ]:
#Create KafkaProducer
def connect_kafka_producer():
    _producer = None
    try: 
       
        _producer = KafkaProducer(bootstrap_servers=[f'{host_ip}:9092'],
                                  value_serializer=lambda x:dumps(x).encode('ascii'),
                                  api_version=(0, 10),
                                 client_id = "hotspotTERRA")
        
    except Exception as ex:
        print("Exception while connecting Kafka")
        print(str(ex))
        
    finally:
        return _producer

#Publish messages
def publish_message(producer_instance, topic_name, key, data):
    try:
        key_bytes = bytes(key, encoding = 'utf-8')
        
        producer_instance.send(topic_name, key=key_bytes, value=data)
        producer_instance.flush()
        
        print("Message successfully publish. Data: " + str(data))
        
    except Exception as ex:
        print('Exception in publishing message.')
        print(str(ex))

In [ ]:
def addDateTime(currDateTime):
    currDateTime = datetime.strptime(currDateTime, '%Y-%m-%dT%H:%M:%S')
    newDateTime = currDateTime + dt.timedelta(hours = 4, minutes=48)
    
    return newDateTime.isoformat()


In [ ]:
topic = "hotspotTopic"

print("Publishing records...")
producer = connect_kafka_producer()
prod_id = producer.config["client_id"]

newDateTime = '2023-01-02T00:00:00'

while True:
    newDateTime = addDateTime(newDateTime)
    randIndex = random.randint(1,len(TERRA_file)-1)
    row = TERRA_file[randIndex]
    
    hotspotTERRA_record = {
        "producer_info": prod_id,
        "datetime": newDateTime,
        "latitude": float(row[0]),
        "longitude": float(row[1]),
        "confidence": int(row[2]),
        "surface_temperature": int(row[3])
    }

    publish_message(producer, topic, 'hotspotTERRA', hotspotTERRA_record)
    sleep(2) #Produce every 2 seconds